In [15]:
import pickle
from sklearn.pipeline import Pipeline
import pandas as pd


In [2]:
def load_model(path) -> Pipeline:
    with open(path, "rb") as f:
        return pickle.load(f)

In [12]:
from classifier.svm_classifier import score_function

In [7]:
from glob import glob
models = glob("regular_grid*")
models

['regular_grid.model',
 'regular_grid_linear.model',
 'regular_grid_linear_c_0.1_10.model',
 'regular_grid_poly_c_100_10000_gamma_0.1_1.model',
 'regular_grid_rbf_c_1000_10000_gamma_0.00001_0.1.model',
 'regular_grid_rbf_c_10_gamma_0.01_0.1_1_100.model',
 'regular_grid_sigmoid_c_100_10000_gamma_0.1_1.model']

In [18]:
global_best_score = 0
global_best_classifier = None
global_best_params = {}
global_results = []
for model_file in models:
    model = load_model(model_file)
    results = model.cv_results_
    best_estimator = model.best_estimator_
    best_score = model.best_score_
    best_params = model.best_params_
    if best_score > global_best_score:
        global_best_score = best_score
        global_best_classifier = best_estimator
        global_best_params = best_params
    global_results.append(pd.DataFrame(results))
merged = pd.concat(global_results)
merged

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ovr__estimator__C,param_ovr__estimator__gamma,param_ovr__estimator__kernel,params,split0_test_correct,split1_test_correct,...,split2_test_time_12_8,split2_test_time_2_2,split2_test_time_2_4,split2_test_time_3_4,split2_test_time_3_8,split2_test_time_4_4,split2_test_time_6_8,split2_test_time_9_8,split2_test_time_common,split2_test_time_cut
0,412.370515,19.066450,316.920705,6.084547,1,1,rbf,"{'ovr__estimator__C': 1, 'ovr__estimator__gamm...",0.202053,0.120477,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1167.660913,4.090325,467.707646,38.368339,0.01,NaN,linear,"{'ovr__estimator__C': 0.01, 'ovr__estimator__k...",0.058933,0.036392,...,0.007463,0.000000,0.000000,0.000000,0.000000,0.000000,0.045113,0.000000,0.000000,0.000000
1,1525.461223,221.765784,570.196996,140.080990,1,NaN,linear,"{'ovr__estimator__C': 1, 'ovr__estimator__kern...",0.143524,0.122862,...,0.000000,0.000000,0.000000,0.000000,0.030075,0.037313,0.308271,0.000000,0.425373,0.000000
2,1872.174747,710.341334,265.527830,118.814240,100,NaN,linear,"{'ovr__estimator__C': 100, 'ovr__estimator__ke...",0.207588,0.201555,...,0.156716,0.060606,0.007519,0.253731,0.353383,0.126866,0.112782,0.308271,0.119403,0.373134
0,1318.331360,16.558714,461.929854,22.232851,0.1,NaN,linear,"{'ovr__estimator__C': 0.1, 'ovr__estimator__ke...",0.100917,0.118040,...,0.000000,0.000000,0.000000,0.000000,0.172932,0.007463,0.578947,0.000000,0.000000,0.000000
1,1157.745227,289.175906,469.635227,252.664433,10,NaN,linear,"{'ovr__estimator__C': 10, 'ovr__estimator__ker...",0.168403,0.119751,...,0.000000,0.000000,0.037594,0.425373,0.000000,0.000000,0.015038,0.000000,0.104478,0.029851
0,2491.958627,29.692040,984.864852,11.019118,100,0.1,poly,"{'ovr__estimator__C': 100, 'ovr__estimator__ga...",0.267144,0.201711,...,0.007463,0.022727,0.000000,0.007463,0.142857,0.022388,0.233083,0.015038,0.014925,0.037313
1,4739.449227,1224.551839,549.055822,264.893611,100,1,poly,"{'ovr__estimator__C': 100, 'ovr__estimator__ga...",0.334163,0.306376,...,0.179104,0.128788,0.225564,0.298507,0.368421,0.320896,0.345865,0.150376,0.238806,0.111940
2,3656.017917,342.592581,664.185412,162.364531,10000,0.1,poly,"{'ovr__estimator__C': 10000, 'ovr__estimator__...",0.321878,0.268274,...,0.186567,0.159091,0.398496,0.119403,0.278195,0.201493,0.225564,0.300752,0.276119,0.097015
3,5522.430417,1401.915754,154.588165,37.768067,10000,1,poly,"{'ovr__estimator__C': 10000, 'ovr__estimator__...",0.360908,0.339191,...,0.179104,0.204545,0.428571,0.216418,0.481203,0.320896,0.345865,0.368421,0.171642,0.082090


In [19]:
merged.to_csv("regular_results.csv")

In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from joblib import Parallel, delayed

def _predict(estimator, X, start, stop):
    return estimator.predict((X[start:stop]))

def confusion_matrices(clf, X, y):
    n_jobs = 32
    n_batches = n_jobs
    n_samples = len(X)
    batch_size = int(np.ceil(n_samples / n_batches))
    parallel = Parallel(n_jobs=n_jobs)
    results = parallel(delayed(_predict)(clf, X, i, i + batch_size)
                       for i in range(0, n_samples, batch_size))
    y_pred = np.concatenate(results)

    c_matrix = confusion_matrix(y, y_pred, normalize='all')
    c_matrix_all = confusion_matrix(y, y_pred, normalize='all')
    c_matrix_true = confusion_matrix(y, y_pred, normalize='true')
    labels = unique_labels(y, y_pred)

    return labels, c_matrix, c_matrix_all, c_matrix_true



In [ ]:
from classifier.svm_preprocessor import load_dataset
dataset = load_dataset('regular.data')
dataset = tuple(dataset)
features = [d[1] for d in dataset]
classes = [d[2] for d in dataset]

labels, cm, cm_all, cm_true = confusion_matrices(global_best_classifier, features, classes)

In [ ]:
pd_cm = pd.DataFrame(data = cm_true, index=labels, columns=labels)
pd_cm
